In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path

notebook_path = Path().absolute()
sys.path.append(str(notebook_path.parent))

In [3]:
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from neural_controllers import NeuralController
import utils

In [4]:
model_type = 'llama'

if model_type=='llama':
    model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

    language_model = AutoModelForCausalLM.from_pretrained(
        model_id, device_map="cuda"
    )

    use_fast_tokenizer = "LlamaForCausalLM" not in language_model.config.architectures
    tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=use_fast_tokenizer, padding_side="left", legacy=False)
    model_name='llama_3_8b_it'
    assistant_tag = '<|start_header_id|>assistant<|end_header_id|>'
    
elif model_type=='gemma':

    tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-9b-it")
    language_model = AutoModelForCausalLM.from_pretrained(
        "google/gemma-2-9b-it",
        device_map="auto",
        torch_dtype=torch.bfloat16,
    )
    model_name='gemma_2_9b_it'
    
tokenizer.pad_token_id = 0 if tokenizer.pad_token_id is None else tokenizer.pad_token_id

2025-03-03 11:44:52.598705: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741023892.819727 1117233 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741023892.892913 1117233 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-03 11:44:53.492185: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
data_dir = "../data/poetry"

dataset = utils.poetry_dataset(data_dir=data_dir, tokenizer=tokenizer, assistant_tag=assistant_tag)

train 200 test 0
train 200 test 0


In [6]:
concept_types = ['prose', 'poetry']

controllers = {}
for concept_type in tqdm(concept_types):
    
    other_type = [k for k in concept_types if k != concept_type][0]
    
    train_data = dataset[concept_type]['train']
    test_data = dataset[concept_type]['test']
        
    controller = NeuralController(
        language_model,
        tokenizer,
        rfm_iters=8,
        batch_size=2,
        control_method='logistic'
    )
    
    controller.compute_directions(train_data['inputs'], train_data['labels'])
    
    controllers[concept_type] = controller
    

  0%|          | 0/2 [00:00<?, ?it/s]

Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : logistic
rfm_iters            : 8
forward_batch_size   : 2
M_batch_size         : 2048
n_components         : 5

use_concat False
Getting activations from forward passes



100%|██████████| 100/100 [00:15<00:00,  6.62it/s]

  0%|          | 0/31 [00:00<?, ?it/s]/u/dbeaglehole/.conda/envs/daniel_jax/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/u/dbeaglehole/.conda/envs/daniel_jax/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/u/dbeaglehole/.conda/envs/daniel_jax/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its d

train X shape: torch.Size([160, 4096]) train y shape: torch.Size([160, 1]) val X shape: torch.Size([40, 4096]) val y shape: torch.Size([40, 1])
Training logistic regression
Logistic probe loss: 7.534022706518853e-06, C: 10, acc: 100.0
beta torch.Size([4096, 1])
concept_features torch.Size([1, 4096])



  3%|▎         | 1/31 [00:01<00:40,  1.33s/it]/u/dbeaglehole/.conda/envs/daniel_jax/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/u/dbeaglehole/.conda/envs/daniel_jax/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/u/dbeaglehole/.conda/envs/daniel_jax/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warni

...

 87%|████████▋ | 27/31 [00:01<00:00, 16.34it/s]/u/dbeaglehole/.conda/envs/daniel_jax/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/u/dbeaglehole/.conda/envs/daniel_jax/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/u/dbeaglehole/.conda/envs/daniel_jax/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warni

train X shape: torch.Size([160, 4096]) train y shape: torch.Size([160, 1]) val X shape: torch.Size([40, 4096]) val y shape: torch.Size([40, 1])
Training logistic regression
Logistic probe loss: 0.11497965633190739, C: 100, acc: 95.0
beta torch.Size([4096, 1])
concept_features torch.Size([1, 4096])
train X shape: torch.Size([160, 4096]) train y shape: torch.Size([160, 1]) val X shape: torch.Size([40, 4096]) val y shape: torch.Size([40, 1])
Training logistic regression
Logistic probe loss: 0.1339614087357019, C: 100, acc: 97.5
beta torch.Size([4096, 1])
concept_features torch.Size([1, 4096])
train X shape: torch.Size([160, 4096]) train y shape: torch.Size([160, 1]) val X shape: torch.Size([40, 4096]) val y shape: torch.Size([40, 1])
Training logistic regression
Logistic probe loss: 0.1572475504872299, C: 1000, acc: 97.5
beta torch.Size([4096, 1])
concept_features torch.Size([1, 4096])
train X shape: torch.Size([160, 4096]) train y shape: torch.Size([160, 1]) val X shape: torch.Size([40, 


100%|██████████| 31/31 [00:01<00:00, 16.08it/s]


Logistic probe loss: 0.19417058756412683, C: 1000, acc: 95.0
beta torch.Size([4096, 1])
concept_features torch.Size([1, 4096])
Computing signs
tensors torch.Size([200, 4096]) direction torch.Size([4096])
tensors torch.Size([200, 4096]) direction torch.Size([4096])
tensors torch.Size([200, 4096]) direction torch.Size([4096])
tensors torch.Size([200, 4096]) direction torch.Size([4096])
tensors torch.Size([200, 4096]) direction torch.Size([4096])
tensors torch.Size([200, 4096]) direction torch.Size([4096])
tensors torch.Size([200, 4096]) direction torch.Size([4096])
tensors torch.Size([200, 4096]) direction torch.Size([4096])
tensors torch.Size([200, 4096]) direction torch.Size([4096])
tensors torch.Size([200, 4096]) direction torch.Size([4096])
tensors torch.Size([200, 4096]) direction torch.Size([4096])
tensors torch.Size([200, 4096]) direction torch.Size([4096])
tensors torch.Size([200, 4096]) direction torch.Size([4096])
tensors torch.Size([200, 4096]) direction torch.Size([4096])
ten


100%|██████████| 2/2 [00:34<00:00, 17.32s/it]


In [9]:
for concept_type in concept_types:
    controller = controllers[concept_type]    
    controller.save(concept=f'{concept_type}', model_name=model_name, path='../directions/')

KeyError: 'prose'

# Control

In [14]:
concept_types = ['prose', 'poetry']

controllers = {}
for concept_type in concept_types:
    
    controller = NeuralController(
        language_model,
        tokenizer,
        control_method='logistic'
    )
    
    other_type = [k for k in concept_types if k!=concept_type][0]
    
    controller.load(concept=f'{concept_type}', model_name=model_name, path='../directions/')
    
    controllers[concept_type] = controller
    

Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : logistic
rfm_iters            : 8
forward_batch_size   : 8
M_batch_size         : 2048
n_components         : 5

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : logistic
rfm_iters            : 8
forward_batch_size   : 8
M_batch_size         : 2048
n_components         : 5

Detector found


/u/dbeaglehole/.conda/envs/daniel_jax/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [26]:
concept_type = "prose"
# concept_type = "poetry"
controller = controllers[concept_type]

raw_inputs = [
    # f"How should I treat a cold?",
    f"What can I buy in a grocery store?",
    # f"What might a student study in school?",
    # f"Tell me about something interesting.",
    # f"Give me advice for applying to jobs.",
]
inputs = [controller.format_prompt(x) for x in raw_inputs]

num_new_tokens = 150

coef=0.7 #llama 
# coef=9

layers = list(range(-1, -31, -1))
# layers = list(range(-1, -41, -1))

gens=[]
print()
for i in inputs:
    print("Prompt:", i)
    # print("===== No Control =====")
    # print(controller.generate(i, max_new_tokens=num_new_tokens, do_sample=False).replace(i, ""))
    # print()
    
    print(f"===== + {concept_type} Control =====")
    gen = controller.generate(i, layers_to_control=layers, control_coef=coef, 
                                max_new_tokens=num_new_tokens, do_sample=False).replace(i, "")
    gens.append(gen)
    print(gen)
    print()
    print()

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Prompt: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

What can I buy in a grocery store?<|eot_id|>
===== + prose Control =====
<|start_header_id|>assistant<|end_header_id|>

Here's a poem inspired by your request:

In twilight's hush, where moonlight whispers low,
A canvas of dreams, I weave for you to know:

In aisles of wonder, where shelves entwine,
A symphony of scents, a tapestry divine:

Fresh blooms of roses, plucked from twilight's sigh,
Their petals like silk, a gentle lover's sigh:

Or perhaps, a sonnet of sonorous delight,
Inspired by the muses, in a midnight's light:

In a market of enchantment, where artisans weave,
A tapestry of taste, a symphony to conceive:

Here's a selection of poems, each a unique offering:

**Poem 1: "Moon


